# Capstone Project 
### David Llerena



## Introduction: Business Problem <a name="introduction"></a>

Several grocery stores are affected by the current pandemic outbreak, the goal of the project is to find those stores that can be close to neighborhoods and that can avoid the crowds in large supermarkets to avoid contagion

## Data <a name="data"></a>

Our information will be:
* Number of stores near the most populated neighborhoods
* Number of grocery stores around the area

In [4]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import csv

In [5]:
pd.set_option('max_colwidth', 800)

In [6]:
source = requests.get('https://es.wikipedia.org/wiki/Categor%C3%ADa:Barrios_de_Guayaquil').text 
soup = BeautifulSoup(source, 'lxml')

In [7]:
csv_file = open('guayaquil.csv', 'w')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Neighbourhood'])

15

In [8]:
mwcg = soup.find_all(class_ = "mw-category-group")

length = len(mwcg) 
for i in range(1, length):  
    lists = mwcg [i].find_all('a')
    for list in lists:
        nbd = list.get('title') 
        csv_writer.writerow([nbd])

In [9]:
csv_file.close()

In [10]:
df = pd.read_csv('guayaquil.csv')

In [11]:
df.shape

(20, 1)

In [12]:
import json

latitudes = [] # Initializing the latitude array
longitudes = [] # Initializing the longitude array

for nbd in df["Neighbourhood"] : 
    place_name = nbd + ",Guayaquil,Ecuador" # Formats the place name
    API_KEY=''
    url = 'https://maps.googleapis.com/maps/api/geocode/json?address={}&key={}'.format(place_name, API_KEY) # Gets the proper url to make the API call
    obj = json.loads(requests.get(url).text) # Loads the JSON file in the form of a python dictionary
    
    results = obj['results'] # Extracts the results information out of the JSON file
    lat = results[0]['geometry']['location']['lat'] # Extracts the latitude value
    lng = results[0]['geometry']['location']['lng'] # Extracts the longitude value
    
    latitudes.append(lat) # Appending to the list of latitudes

IndexError: list index out of range

In [ ]:
df['Latitude'] = latitudes
df['Longitude'] = longitudes

In [ ]:
df.head()

In [ ]:
col = 0
explored_lat_lng = []
for lat, lng, neighbourhood in zip(df['Latitude'], df['Longitude'], df['Neighbourhood']):
    if (lat, lng) in explored_lat_lng:
        col = col + 1
    else:
        explored_lat_lng.append((lat, lng))

print("Collisions : ", col)



In [ ]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium # Map plotting library
import numpy as np
from pandas.io.json import json_normalize # Tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# Import k-means from clustering stage
from sklearn.cluster import KMeans

In [ ]:
gua_lat = 22.5726
gua_lng = 88.3639

# Creates map of Kolkata using latitude and longitude values
map_gua = folium.Map(location=[gua_lat, gua_lng], zoom_start=10)

# Add markers to map
for lat, lng, neighbourhood in zip(df['Latitude'], df['Longitude'], df['Neighbourhood']):
    label = '{}'.format(neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_gua)  
    
map_gua

In [ ]:
def get_category_type(row):
    categories_list = row['Category']  
    
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [ ]:


explore_df_list = []

for i, nbd_name in enumerate(df['Neighbourhood']):  
    
    try :
        ### Getting the data of neighbourhood
        nbd_name = df.loc[i, 'Neighbourhood']
        nbd_lat = df.loc[i, 'Latitude']
        nbd_lng = df.loc[i, 'Longitude']

        radius = 1000 # Setting the radius as 1000 metres
        LIMIT = 30 # Getting the top 30 venues

        url = 'https://api.foursquare.com/v2/venues/explore?client_id={} \
        &client_secret={}&ll={},{}&v={}&radius={}&limit={}'\
        .format(CLIENT_ID, CLIENT_SECRET, nbd_lat, nbd_lng, VERSION, radius, LIMIT)

        results = json.loads(requests.get(url).text)
        results = results['response']['groups'][0]['items']

        nearby = json_normalize(results) # Flattens JSON

        # Filtering the columns
        filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
        nearby = nearby.loc[:, filtered_columns]

        # Renaming the columns
        columns = ['Name', 'Category', 'Latitude', 'Longitude']
        nearby.columns = columns

        # Gets the categories
        nearby['Category'] = nearby.apply(get_category_type, axis=1)    

        # Gets the data required
        for i, name in enumerate(nearby['Name']):
            s_list = nearby.loc[i, :].values.tolist()  # Converts the numpy array to a python list
            f_list = [nbd_name, nbd_lat, nbd_lng] + s_list
            explore_df_list.append(f_list)
    
    except Exception as e:
        pass



In [ ]:
explore_df = pd.DataFrame([item for item in explore_df_list])
explore_df.columns = ['Neighbourhood', 'Neighbourhood Latitude', 'Neighbourhood Longitude', 'Venue Name', 'Venue Category', 'Venue Latitude', 'Venue Longitude']
explore_df.head()